In [9]:
import jax.numpy as jnp
import jax
from jax import Array
import jax_dataclasses as jdc
from jaxlie import *

In [169]:
d = jnp.zeros(3)
def distance(d):
    is_zero = jnp.allclose(d, 0.)
    d = jnp.where(is_zero, jnp.ones_like(d), d)
    l = jnp.linalg.norm(d)
    l = jnp.max(jnp.array([l, 1e-6]))
    return l

@jdc.pytree_dataclass
class circle:
    center: Array
    r: int
    def penetration(self, points):
        d = jax.vmap(distance)(points - self.center) - self.r
        return - (d)

@jdc.pytree_dataclass
class EnvSDF:
    sdfs: tuple
    def penetration(self, points, num_points:int):
        result = jnp.zeros(num_points)
        for sdf in self.sdfs:
            result = jnp.maximum(sdf.penetration(points), result)
        return result

@jax.jit
def robot_penetration_cost(points, len_points, sdf: circle, safe_dist):
    d = sdf.penetration(points, )
    return d + safe_dist

In [175]:
circ = circle(jnp.array([1,0,0.]), 1.)
env = EnvSDF((circ))

In [178]:
robot_points = jnp.array([
    [-0., 0, 0.],
    [-0.3, 0, 0.],
    [-0.4, 3, 0.]
])
robot_penetration_cost(robot_points, env, 0.3)

TypeError: penetration() missing 1 required positional argument: 'num_points'

In [150]:
p = jnp.array([0.,0,0])
jax.grad(penetration)(p, circ, 0.)

Array([ 1., -0., -0.], dtype=float32)